In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import os

base_dir = os.getcwd()  
parent_dir = os.path.dirname(base_dir)    
csv_path = {0: os.path.join(parent_dir, "RWD_20250422/D1T4", "observation_data_case5.csv"),
            1: os.path.join(parent_dir, "for_real", "Case1_Urban_10Hz.csv"),
            2: os.path.join(parent_dir, "for_real", "Case1_Suburban_10Hz.csv"),
            3: os.path.join(parent_dir, "for_real", "Case2_Urban_10Hz.csv"),
            4: os.path.join(parent_dir, "for_real", "Case2_Suburban_10Hz.csv"),
            5: os.path.join(parent_dir, "for_real", "Case3_Urban_10Hz.csv"),
            6: os.path.join(parent_dir, "for_real", "Case3_Suburban_10Hz.csv"),
            7: os.path.join(parent_dir, "for_real", "Case4_Urban_10Hz.csv"),
            8: os.path.join(parent_dir, "for_real", "Case4_Suburban_10Hz.csv"),
            }


data = {}
for key in csv_path.keys():
    # data[key] = pd.read_csv(csv_path[key])
    # print('\n Case', key, ': Length of data =', len(data[key]['gps_time']), ', Duration =', data[key]['gps_time'].iloc[-1] - data[key]['gps_time'].iloc[0], 's, columns =', data[key].columns.tolist())
    # print(data[key].head())
    
    df = pd.read_csv(csv_path[key])
    length = len(df)
    print(length)
    duration = df['gps_time'].iloc[-1] - df['gps_time'].iloc[0] if length > 1 else 0
    columns = df.columns.tolist()
    # Type masks
    pl_mask = df['satellite_id'].str.startswith("P")
    pl_total = pl_mask.sum()
    gps_mask = df['satellite_id'].str.startswith("G")
    gps_total = gps_mask.sum()
    print(gps_total)
    # Multipath labels
    if key == 0:
        total_los = total_nlos = pl_los = pl_nlos = gps_los = gps_nlos = 0
    else:
        total_los  = (df['multipath'] == 0).sum()
        total_nlos = (df['multipath'] == 1).sum()
        
        pl_los  = ((pl_mask) & (df['multipath'] == 0)).sum()
        pl_nlos = ((pl_mask) & (df['multipath'] == 1)).sum()

        gps_los  = ((gps_mask) & (df['multipath'] == 0)).sum()
        gps_nlos = ((gps_mask) & (df['multipath'] == 1)).sum()

    # Print formatted summary
    print(f"\n📦 Case {key}: {os.path.basename(csv_path[key])}")
    print(f"  ▶ Length     : {length} rows")
    print(f"  ▶ Duration   : {duration:.2f} s")
    print(f"  ▶ Columns    : {len(columns)} → {columns}")
    print(f"  ▶ PL         : {pl_total} (LOS: {pl_los}, NLOS: {pl_nlos}) (LOS %: {pl_los/pl_total*100:.1f}%, NLOS %: {pl_nlos/pl_total*100:.1f}%)")
    print(f"  ▶ GPS        : {gps_total} (LOS: {gps_los}, NLOS: {gps_nlos}) (LOS %: {gps_los/gps_total*100:.1f}%, NLOS %: {gps_nlos/gps_total*100:.1f}%)")
    if key != 0:
        print(f"  ▶ Total SIG  : LOS: {total_los}, NLOS: {total_nlos} (LOS %: {total_los/(total_los+total_nlos)*100:.1f}%, NLOS %: {total_nlos/(total_los+total_nlos)*100:.1f}%)")

#这个 里面是把所有的数据文件中的全体信号数量占比打印出来 


12614
5788

📦 Case 0: observation_data_case5.csv
  ▶ Length     : 12614 rows
  ▶ Duration   : 52.50 s
  ▶ Columns    : 13 → ['gps_time', 'satellite_id', 'sv_id', 'pseudorange', 'doppler_shift', 'cn0', 'azimuth', 'elevation', 'pseudorange_residual', 'pseudorange_corrected_cb', 'sat_px', 'sat_py', 'sat_pz']
  ▶ PL         : 1394 (LOS: 0, NLOS: 0) (LOS %: 0.0%, NLOS %: 0.0%)
  ▶ GPS        : 5788 (LOS: 0, NLOS: 0) (LOS %: 0.0%, NLOS %: 0.0%)
91719
55773

📦 Case 1: Case1_Urban_10Hz.csv
  ▶ Length     : 91719 rows
  ▶ Duration   : 599.00 s
  ▶ Columns    : 12 → ['gps_time', 'satellite_id', 'sv_id', 'pseudorange', 'doppler_shift', 'cn0', 'rec_pow', 'azimuth', 'elevation', 'pseudorange_residual', 'pseudorange_corrected_cb', 'multipath']
  ▶ PL         : 35946 (LOS: 29912, NLOS: 6034) (LOS %: 83.2%, NLOS %: 16.8%)
  ▶ GPS        : 55773 (LOS: 10853, NLOS: 44920) (LOS %: 19.5%, NLOS %: 80.5%)
  ▶ Total SIG  : LOS: 40765, NLOS: 50954 (LOS %: 44.4%, NLOS %: 55.6%)
91719
55773

📦 Case 2: Case1_Sub

In [ ]:
# Define cases
real_case = 0
# training_data = [1,2,3,4,5,6]
# testing_data = [7,8]
training_data = [2,3]
testing_data = [4]


# Load real case
df_real = pd.read_csv(csv_path[real_case])

# Load training cases into one DataFrame
df_train = pd.concat(
    [pd.read_csv(csv_path[k]).assign(case_id=k) for k in training_data],
    ignore_index=True
)

# Load testing cases into one DataFrame
df_test = pd.concat(
    [pd.read_csv(csv_path[k]).assign(case_id=k) for k in testing_data],
    ignore_index=True
)

# Classify satellite types in all DataFrames (PL=1, GPS=0, OTHER=-1)
for df in [df_real, df_train, df_test]:
    df['satellite_id'] = df['satellite_id'].astype(str)
    df['sat_type'] = np.where(
        df['satellite_id'].str.contains("^P", na=False), 1, # ^ means “start of string”.
        np.where(df['satellite_id'].str.contains("^G", na=False), 0, -1)
    )

print("Any real data OTHER than PL/GPS?", (df_real['sat_type'] == -1).any())
print("Any train data OTHER than PL/GPS?", (df_train['sat_type'] == -1).any())
print("Any test data OTHER than PL/GPS?", (df_test['sat_type'] == -1).any())

Any real data OTHER than PL/GPS? True
Any train data OTHER than PL/GPS? False
Any test data OTHER than PL/GPS? False


In [3]:
for col in ["doppler_shift", "elevation", "pseudorange_residual", "pseudorange_corrected_cb", "cn0"]:
    print(f"\n📊 Feature: {col}")
    for sat_type in [0, 1]:  # 0 = GPS, 1 = PL
        real_mask = df_real["sat_type"] == sat_type
        train_mask = df_train["sat_type"] == sat_type

        real_mean = df_real.loc[real_mask, col].mean()
        train_mean = df_train.loc[train_mask, col].mean()
        diff = real_mean - train_mean

        print(f"  sat_type {sat_type}: Real mean = {real_mean:.4f}, Train mean = {train_mean:.4f}, Diff = {diff:.4f}")



📊 Feature: doppler_shift
  sat_type 0: Real mean = -191.2868, Train mean = -27.4664, Diff = -163.8204
  sat_type 1: Real mean = 52.1876, Train mean = -0.0011, Diff = 52.1887

📊 Feature: elevation
  sat_type 0: Real mean = 43.2567, Train mean = 35.3412, Diff = 7.9155
  sat_type 1: Real mean = 4.4673, Train mean = 15.5046, Diff = -11.0373

📊 Feature: pseudorange_residual
  sat_type 0: Real mean = -0.3730, Train mean = -0.3427, Diff = -0.0304
  sat_type 1: Real mean = 1.2762, Train mean = -0.3049, Diff = 1.5811

📊 Feature: pseudorange_corrected_cb
  sat_type 0: Real mean = 22186348.1537, Train mean = 22612315.6011, Diff = -425967.4474
  sat_type 1: Real mean = 26.7248, Train mean = 22.5770, Diff = 4.1478

📊 Feature: cn0
  sat_type 0: Real mean = 43.6147, Train mean = 36.2806, Diff = 7.3341
  sat_type 1: Real mean = 47.7305, Train mean = 167.2583, Diff = -119.5279


In [4]:

# --- Compute real-world medians for PL and GPS ---
real_pl_med  = df_real.loc[df_real['sat_type'] == 1, 'cn0'].median()
real_gps_med = df_real.loc[df_real['sat_type'] == 0, 'cn0'].median()

# --- Align cn0 in training data ---
train_pl_med  = df_train.loc[df_train['sat_type'] == 1, 'cn0'].median()
train_gps_med = df_train.loc[df_train['sat_type'] == 0, 'cn0'].median()

diff_pl  = real_pl_med  - train_pl_med
diff_gps = real_gps_med - train_gps_med

df_train['cn0_new'] = df_train['cn0']
df_test['cn0_new'] = df_test['cn0']
df_train.loc[df_train['sat_type'] == 1, 'cn0_new'] += diff_pl
df_train.loc[df_train['sat_type'] == 0, 'cn0_new'] += diff_gps
df_test.loc[df_test['sat_type'] == 1, 'cn0_new'] += diff_pl
df_test.loc[df_test['sat_type'] == 0, 'cn0_new'] += diff_gps

print(f"Real PL median: {real_pl_med}, Train PL median: {train_pl_med}, Diff: {diff_pl}")
print(f"Real GPS median: {real_gps_med}, Train GPS median: {train_gps_med}, Diff: {diff_gps}")
print(f"After adjustment, New Train PL median: {df_train.loc[df_train['sat_type'] == 1, 'cn0_new'].median()}")
print(f"After adjustment, New Train GPS median: {df_train.loc[df_train['sat_type'] == 0, 'cn0_new'].median()}")



Real PL median: 48.8056396296138, Train PL median: 170.05416582270487, Diff: -121.24852619309107
Real GPS median: 45.4344222328504, Train GPS median: 42.0615650724157, Diff: 3.372857160434698
After adjustment, New Train PL median: 48.80563962961381
After adjustment, New Train GPS median: 45.434422232850395


In [5]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

# ----------------------------
# 1. Dataset Preparation
# ----------------------------
features = ["cn0_new", "elevation", "pseudorange_residual", "sat_type"]
target = "multipath"
SEQ_LEN = 10  # sequence length (10 epochs per satellite)

# Encode sv_id
le = LabelEncoder()
df_train["sv_id_enc"] = le.fit_transform(df_train["sv_id"])
df_test["sv_id_enc"]  = le.transform(df_test["sv_id"])
features_with_id = features + ["sv_id_enc"]

def make_sequences(df, seq_len):
    X, y = [], []
    for sat_id, sat_df in df.groupby("sv_id"):
        sat_df = sat_df.sort_values("gps_time")
        arr = sat_df[features_with_id].values
        labels = sat_df[target].values
        for i in range(len(sat_df) - seq_len + 1):
            X.append(arr[i:i+seq_len])
            y.append(labels[i+seq_len-1])  # label = last in window
    return np.array(X, dtype=np.float32), np.array(y, dtype=np.int64)

X_train, y_train = make_sequences(df_train, SEQ_LEN)
X_test, y_test   = make_sequences(df_test, SEQ_LEN)

print("Train shape:", X_train.shape, y_train.shape)
print("Test shape:", X_test.shape, y_test.shape)

# ----------------------------
# 2. PyTorch Dataset + DataLoader
# ----------------------------
class SeqDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X)
        self.y = torch.tensor(y)
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_loader = DataLoader(SeqDataset(X_train, y_train), batch_size=64, shuffle=True)
test_loader  = DataLoader(SeqDataset(X_test, y_test), batch_size=64, shuffle=False)

# ----------------------------
# 3. LSTM Model
# ----------------------------
class LSTMClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim=64, num_layers=1, num_classes=2):
        super(LSTMClassifier, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc1 = nn.Linear(hidden_dim, 32)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.3)
        self.fc2 = nn.Linear(32, num_classes)

    def forward(self, x):
        out, _ = self.lstm(x)               # out: [batch, seq, hidden]
        out = out[:, -1, :]                 # take last time step
        out = self.fc1(out)
        out = self.relu(out)
        out = self.dropout(out)
        out = self.fc2(out)
        return out

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LSTMClassifier(input_dim=X_train.shape[2]).to(device)

# ----------------------------
# 4. Training Loop
# ----------------------------
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

EPOCHS = 10
for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch [{epoch+1}/{EPOCHS}], Loss: {total_loss/len(train_loader):.4f}")

# ----------------------------
# 5. Evaluation
# ----------------------------
model.eval()
y_true, y_pred = [], []
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch = X_batch.to(device)
        outputs = model(X_batch)
        preds = torch.argmax(outputs, dim=1).cpu().numpy()
        y_true.extend(y_batch.numpy())
        y_pred.extend(preds)

print("\n=== Classification Report ===")
print(classification_report(y_true, y_pred, target_names=["LOS (0)", "NLOS (1)"]))


Train shape: (171431, 10, 5) (171431,)
Test shape: (79750, 10, 5) (79750,)
Epoch [1/10], Loss: 0.2691
Epoch [2/10], Loss: 0.2413
Epoch [3/10], Loss: 0.2347
Epoch [4/10], Loss: 0.2281
Epoch [5/10], Loss: 0.2118
Epoch [6/10], Loss: 0.1826
Epoch [7/10], Loss: 0.1726
Epoch [8/10], Loss: 0.1658
Epoch [9/10], Loss: 0.1616
Epoch [10/10], Loss: 0.1580

=== Classification Report ===
              precision    recall  f1-score   support

     LOS (0)       0.91      0.97      0.94     35189
    NLOS (1)       0.97      0.93      0.95     44561

    accuracy                           0.95     79750
   macro avg       0.94      0.95      0.95     79750
weighted avg       0.95      0.95      0.95     79750



In [6]:
# ----------------------------
# 1. Ensure cn0_new exists      real dataset import
# ----------------------------
if "cn0_new" not in df_real.columns and "cn0" in df_real.columns:
    df_real["cn0_new"] = df_real["cn0"]

df_real["sv_id_enc"] = -1  # default for unknown satellites
known_mask = df_real["sv_id"].isin(le.classes_)
df_real.loc[known_mask, "sv_id_enc"] = le.transform(df_real.loc[known_mask, "sv_id"])

# 3. Now define features_with_id (must include sv_id_enc)
features_with_id = ["cn0_new", "elevation", "pseudorange_residual", "sat_type", "sv_id_enc"]

# 4. Remove rows with missing values
df_real_filtered = df_real.dropna(subset=features_with_id).copy()

# Encode sv_id (must use the SAME encoder as training)
# df_real_filtered["sv_id_enc"] = le.transform(df_real_filtered["sv_id"])

# ----------------------------
# 2. Build sequences (no target)
# ----------------------------
def make_sequences_inference(df, seq_len):
    seqs, idx_map = [], []  # keep track of row indices for mapping back
    for sat_id, sat_df in df.groupby("sv_id"):
        sat_df = sat_df.sort_values("gps_time")
        arr = sat_df[features_with_id].values
        for i in range(len(sat_df) - seq_len + 1):
            seqs.append(arr[i:i+seq_len])
            idx_map.append(sat_df.index[i+seq_len-1])  # map prediction to last row
    return np.array(seqs, dtype=np.float32), idx_map

X_real_seq, idx_map = make_sequences_inference(df_real_filtered, SEQ_LEN)

# ----------------------------
# 3. Run LSTM model inference
# ----------------------------
model.eval()
y_real_pred = []
with torch.no_grad():
    X_tensor = torch.tensor(X_real_seq).to(device)
    outputs = model(X_tensor)
    preds = torch.argmax(outputs, dim=1).cpu().numpy()
    y_real_pred.extend(preds)

# ----------------------------
# 4. Add predictions back
# ----------------------------
df_real_filtered["predicted_multipath"] = np.nan  # init column
for idx, pred in zip(idx_map, y_real_pred):
    df_real_filtered.loc[idx, "predicted_multipath"] = pred  # 0=LOS, 1=NLOS

print(f"Predicted {np.sum(df_real_filtered['predicted_multipath']==0)} LOS "
      f"and {np.sum(df_real_filtered['predicted_multipath']==1)} NLOS in real data.")

# ----------------------------
# 5. Per satellite type summary
# ----------------------------
for sat_label, sat_name in zip([0, 1], ["GPS", "PL"]):
    df_sat = df_real_filtered[df_real_filtered["sat_type"] == sat_label]
    los_count = (df_sat["predicted_multipath"] == 0).sum()
    nlos_count = (df_sat["predicted_multipath"] == 1).sum()
    total = df_sat["predicted_multipath"].notna().sum()

    los_percent = 100 * los_count / total if total > 0 else 0
    nlos_percent = 100 * nlos_count / total if total > 0 else 0

    print(f"\n🔎 {sat_name} Signals:")
    print(f"  ▶ Total           : {total}")
    print(f"  ▶ Predicted LOS   : {los_count} ({los_percent:.1f}%)")
    print(f"  ▶ Predicted NLOS  : {nlos_count} ({nlos_percent:.1f}%)")


Predicted 1653 LOS and 9419 NLOS in real data.

🔎 GPS Signals:
  ▶ Total           : 4910
  ▶ Predicted LOS   : 229 (4.7%)
  ▶ Predicted NLOS  : 4681 (95.3%)

🔎 PL Signals:
  ▶ Total           : 1376
  ▶ Predicted LOS   : 230 (16.7%)
  ▶ Predicted NLOS  : 1146 (83.3%)


In [7]:
df_real_filtered["predicted_multipath"] = np.nan  # init column
for idx, pred in zip(idx_map, y_real_pred):
    df_real_filtered.loc[idx, "predicted_multipath"] = pred  # 0=LOS, 1=NLOS

print(f"Predicted {np.sum(df_real_filtered['predicted_multipath']==0)} LOS "
      f"and {np.sum(df_real_filtered['predicted_multipath']==1)} NLOS in real data.")

print(df_real_filtered['predicted_multipath'])
# ----------------------------
# 5. Per satellite type summary
# ----------------------------
for sat_label, sat_name in zip([0, 1], ["GPS", "PL"]):
    df_sat = df_real_filtered[df_real_filtered["sat_type"] == sat_label]
    los_count = (df_sat["predicted_multipath"] == 0).sum()
    nlos_count = (df_sat["predicted_multipath"] == 1).sum()
    total = df_sat["predicted_multipath"].notna().sum()

    los_percent = 100 * los_count / total if total > 0 else 0
    nlos_percent = 100 * nlos_count / total if total > 0 else 0

    print(f"\n🔎 {sat_name} Signals:")
    print(f"  ▶ Total           : {total}")
    print(f"  ▶ Predicted LOS   : {los_count} ({los_percent:.1f}%)")
    print(f"  ▶ Predicted NLOS  : {nlos_count} ({nlos_percent:.1f}%)")

Predicted 1653 LOS and 9419 NLOS in real data.
0        NaN
1        1.0
2        NaN
3        NaN
4        1.0
        ... 
12609    1.0
12610    1.0
12611    0.0
12612    0.0
12613    0.0
Name: predicted_multipath, Length: 11243, dtype: float64

🔎 GPS Signals:
  ▶ Total           : 4910
  ▶ Predicted LOS   : 229 (4.7%)
  ▶ Predicted NLOS  : 4681 (95.3%)

🔎 PL Signals:
  ▶ Total           : 1376
  ▶ Predicted LOS   : 230 (16.7%)
  ▶ Predicted NLOS  : 1146 (83.3%)


In [8]:
# Save to CSV
df_real_filtered.to_csv("real_with_predicted_multipath_LSTM.csv", index=False)
print("✅ Saved: CSV")
#这个是使用 lstm模型跑跑完的真实数据分类结果，所以 我们需要把结果输入回位置解算 然后 和 定位对比 

✅ Saved: CSV
